**Odds-and-ends**

* Allow (datum, colormap) color specifications for all plots.
* Matrix visualization should use CellMark for its implemention (so the data is in the cells).
* Better / more consistent padding when autoplotting.
    * Padding above / below bars.
    * Padding between bars.
    * Padding above / below line plots.
* Allow the grid to be visible when autoplotting.
* Do a better job updating gaps when we insert / delete cells.
* Should grid lines be rendered down the middle of a gap (as we do today), or to either side?

In [1]:
import numpy
import toyplot.data

import logging
toyplot.log.setLevel(logging.DEBUG)

In [2]:
numpy.random.seed(1234)

canvas, table = toyplot.table(trows=3, rows=3, brows=3, lcolumns=3, columns=3, rcolumns=3, filename="rainbow")
table.cells.cells.data = numpy.arange(81).reshape(9, 9)
#table.cells.column[3:6].data = numpy.random.normal(loc=5, size=(9, 3))
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
table.top.left.cells.style = {"fill":"red", "opacity":0.3}
table.top.cells.style = {"fill":"orange", "opacity":0.3}
table.top.right.cells.style = {"fill":"yellow", "opacity":0.3}
table.right.cells.style = {"fill":"greenyellow", "opacity":0.3}
table.bottom.right.cells.style = {"fill":"green", "opacity":0.3}
table.bottom.cells.style = {"fill":"aqua", "opacity":0.3}
table.bottom.left.cells.style = {"fill":"blue", "opacity":0.3}
table.left.cells.style = {"fill":"purple", "opacity":0.3}

#table.cells.row[0:3].height = 25
#table.cells.row[4].height = "1in"
#table.cells.column[4].width = "1in"
#table.cells.gaps.rows[7] = 10

axes = table.cells.column[3:6].cartesian()
#axes.bars(numpy.random.normal(loc=5, size=(20, 2)), along="y", opacity=0.5)
#axes.cell_bars(width=1.0, series="rows")
#axes.vlines(60)
axes.cell_plot(marker="o")

0 1 2 6 7 8 9 10 11 15 16 17 18 19 20 24 25 26 27 28 29 33 34 35 36 37 38 42 43 44 45 46 47 51 52 53 54 55 56 60 61 62 63 64 65 69 70 71 72 73 74 78 79 80 
 
 
 Save as .csv

In [3]:
import toyplot.pdf
toyplot.pdf.render(canvas, "test.pdf")